# Chapter 3 .  Losses used in Style Transfer

## Unit 1.  Losses 

In the previous chapter we were able to extract feature maps from different layers of the pretrained VGG-19 network. Now we move further to the specifics of Neural Style Transfer. From Chapter 1, we learnt that style tranfser uses 2 kinds of images **Content Image** and **Style Image**. Losses play an important role is helping us achieve style transfer using the images. 

The concept of **loss function** is not new to anyone who is working with machine learning. Loss functions help us evaluate how well a specific algorithm learns/performs on the given data. The goal is always to minimize this loss function. If predictions deviates too much from actual labels, loss function grows large to penalize the step. Gradually, with the help of optimization functions, the loss reduces. The loss function used here will be a combination of multiple loss functions, each penalizing specific inputs. 

Style transfer uses 3 loss functions : 

1. **Content Loss** which describes how different the content of two images are
2. **Style Loss** which describes the difference between the two images in terms of their style
3. **Total Variance Loss** which helps to reduce the noise in the generated image. The noise in generic terms includes the common losses in Digital Image Processing ie., Gaussian noise, salt and pepper noise, speckle noise . 
  
Let us now discuss the Loss Functions in detail. Please note that we will be calling the output image as **Generated Image (G) ** since it will be the fusion of content and style images.


## Unit 2. Content Loss (Lc)

As we saw in Chapter 1, the earlier (shallower) layers of the network tend to detect lower-level features such as edges and simple textures, and the latter (deeper) layers tend to detect higher-level features such as objects, shapes, outlines etc. 

We would like the generated image **G** to have similar content as the input image **C**. For achieving this we choose a layer's activations to represent the content of an image. In practice, for content we choose the deeper layers in order to efficiently capture the objects and outlines which form the content of the image. However be careful to not choose the very final layers since they may get too specific for the objects.

Given a chosen content layer **l**, the content loss is defined as the **Mean Squared Error (Euclidean distance)** between the feature map **F** of our content image C and the feature map **P** of our generated image G.

The content loss gives the distance between outputs of the model for the content image and the generated image at a specific layer in the network. Lesser the content loss, more similar are the feature activations for the generated image to content image. The kinds of features that are transferred from content image to the composite image is dependent on the layers we choose.

The content loss can be formulated as follows : 


![](https://bitbucket.org/ga_learning/style_transfer/raw/48db127941e04f2f66cdfd6a3ccaca9ed888d598/markdown_images/Content_loss.png)




where , 

* **L<sub>content</sub>** is the Content Loss 

* **l** is the layer from which the feature maps are obtained

* **i** refers to the index of each feature maps from layer l

* **j** refers to each element in the flattened feature matrix of size h x w

* **F** is the feature map from the Content image (C) obtained at layer l

* **P** is the feature map from the Generated image (G) obtained at layer l 


In [1]:
def content_loss(content, gen):
    
    """ content -->  feature map of the content image
        gen     -->  feature map of the generated image  """
    
    assert K.ndim(content) == 3
    assert K.ndim(gen) == 3
    # Euclidean distance
    return K.sum(K.square(gen - content))

## Unit 3. Style Loss

Style is not very objective like the Content which can be represented by specific edges or lines or objects. Then, how do we measure the style similarity between the style image **S** and the generated image **G**?

Hence before delving in to the Style Loss, we will first understand a very simple yet immensely useful metric called **Gram Matrix**.

### 3.1 Gram Matrix (G<sub>ij</sub>)

The Gram-matrix (**Gij**) is a concept from linear algebra which is a matrix of dot-products between the vectors of the feature activations of a specific layer. In simple terms, Gram-matrix is the dot product of the flattened feature map and the transpose of the flattened feature map both taken at the the same layer.

If an entry in the Gram-matrix has a value close to zero then it means the two features in the given layer do not activate simultaneously for the given image. If an entry in the Gram-matrix has a large value, then it means the two features do activate simultaneously for the given image. Thus Gram matrix gives us the features 

![](https://bitbucket.org/ga_learning/style_transfer/raw/48db127941e04f2f66cdfd6a3ccaca9ed888d598/markdown_images/Gram_matrix.png)



where, 

* **G<sub>ij</sub>** is the Gram matrix 

* **i** refers to the number of feature maps at layer l

* **k** refers to each element in the flattened feature matrix of size h x w

* **F** is the feature map from the Content image (C) obtained at selected layer l



Well, how does Gram matrix indicate the style of the image?  A very important part of the gram matrix is that the diagonal elements (**G<sub>ii</sub>**) measures how active the filters are. For example, suppose filter i is detecting vertical textures in the image. Then **G<sub>ii</sub>** measures how common vertical textures are in the image as a whole. If **G<sub>ii</sub>** is huge, this means that the image has a lot of vertical texture. Same goes for horizontal,angled textures too.

The following diagram is a visual representation of the elements of a gram matrix. Observe the diagonal elements. Both halves are of same colour indicating the product of same features. This is just a simple visualisation. In the actual case we see lots of bright and dark regions which indicate product of activations and mostly the features producing the high values in the Gram Matrix are the ones which help us preserve the style. The following diagram has been taken from a wonderful implementation of Style Transfer [https://datascience-enthusiast.com/DL/Art_Generation_with_Neural_Style_Transfer_v2.html]


![](https://bitbucket.org/ga_learning/style_transfer/raw/48db127941e04f2f66cdfd6a3ccaca9ed888d598/markdown_images/GramMatrix_example.png)




Thus by observing the strength of different types of features (**G<sub>ii</sub>**), as well as how different features occur together (**G<sub>ij</sub>**), the Gram matrix measures the style of an image. Hence it is also known as **Style Matrix**. 

In [2]:
def gram_matrix(x):
    assert K.ndim(x) == 3
    if K.image_data_format() == 'channels_first':
        features = K.flatten(x)
    else:
        features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    # Dot product of the flattened feature map and the transpose of the
    # flattened feature map
    gram = K.dot(features, K.transpose(features))
    return gram

### 3.2 Formulating Style Loss (Ls)

In order to measure style similarity or difference between the style image (S) and the generated image (G), 
we need to obtain the simultaneously activating features in the selected style-layers for the style image as well as generated image. The Gram-matrix or Style-matrix helps us identify the simultaneously activated features.

Once the style is extracted, the Style Loss function is quite similar to the Content loss, except that we calculate the **Mean Squared Error** for the Gram-matrices instead of direct outputs from the layers.

![](https://bitbucket.org/ga_learning/style_transfer/raw/48db127941e04f2f66cdfd6a3ccaca9ed888d598/markdown_images/Style_loss.png)



where, 

* **L<sub>style</sub>** is the Style Loss 

* **l** is the layer from which the feature maps are obtained

* **i** refers to the index of each feature maps from layer l

* **j** refers to each element in the flattened feature matrix of size h x w

* **A<sub>ij</sub>** is the Gram matrix for the Input Image features

* **G<sub>ij</sub>** is the Gram matrix for the Generated Image features

If we had two images whose feature maps at a given layer produced the same Gram matrix we would expect both images to have the same style, but not necessarily the same content. 

Applying this to early layers in the network would capture some of the finer textures contained within the image whereas applying this to deeper layers would capture more higher-level elements of the image’s style. The authors of the paper : A Neural Algorithm of Artistic Styler (gatys et al) found that the best results were achieved by taking a combination of shallow and deep layers as the style representation for an image.

In [3]:
def style_loss(style, gen):
    assert K.ndim(style) == 3
    assert K.ndim(gen) == 3
    S = gram_matrix(style)
    G = gram_matrix(gen)
    channels = 3
    size = img_h * img_w
    # Euclidean distance of the gram matrices multiplied by the constant
    return K.sum(K.square(S - G)) / (4. * (channels ** 2) * (size ** 2))

## Unit 4.  Total Loss

Now that we have learnt about the losses involved, let us look at how to use them in Neural Style Transfer. 
From the above sections we know that the each of the losses control specific features in the output image. The Content Loss  ( **L<sub>content</sub>** ) controls the presence of the content specific features like objects, outiles etc. in the output image and the Style Loss ( **L<sub>style</sub>** ) controls the style specific information like colours, brush strokes etc.  

Putting this all together, the total loss ( **L<sub>total</sub>** ) can be the sum of the 2 losses. However, there arises a need to control how much of the content and how much of the style needs to be present in the output or generated image. Hence we add weights to the total loss. So the total loss is a weighted sum of the style and content losses.

![](https://bitbucket.org/ga_learning/style_transfer/raw/48db127941e04f2f66cdfd6a3ccaca9ed888d598/markdown_images/Total_loss.png)

where, 

* **alpha** is the Content weight  
* **beta**  is the Style weight
* **L<sub>total</sub>** is the Total Loss 
* **L<sub>content</sub>** is the Content Loss 
* **L<sub>style</sub>** is the Style Loss 


In [4]:
def total_loss(Lc , Ls , alpha = 0.5 , beta = 0.5):
    Lt = (alpha*Lc) + (beta*Ls)
    return Lt

## Unit 5. Total Variance Loss

Most of the time, the generated images have a lot of high-frequency noise, particularly bright or dark pixels. To make the output smoother and cleaner we need to suppress this noise. One very effective noise reduction method is **Total Variation Denoising**. This loss is not present in the original paper, however it helps produce better results since it induces smoothness of colours between close pixels and hence widely used.

There are lot of simple denoising methods like mean denoising, median denoising etc. However, the total variation denoising is remarkably effective at simultaneously preserving edges whilst smoothing away noise in flat regions, even at low signal-to-noise ratios. 

It uses the gradients of the image in x and y direction and combines it to provide the smoothing effect. Since it operates on the first order gradients, it helps in preserving the strong edges and also makes the pixels similar to the neighbouring pixels thereby producing the smoothing effect.  

The total variance loss can be represented as **L<sub>tv</sub>**

**L<sub>tv</sub> = Elementwise_Sum( (G<sub>x</sub><sup>2</sup> + G<sub>y</sub><sup>2</sup>) <sup>1.25</sup> )**

where,

* G<sub>x</sub> = Gradient of Image along x-axis
* G<sub>y</sub> = Gradient of Image along y-axis

Below is an example of the smoothing effect obtained using total variation loss (ref: https://en.wikipedia.org/wiki/Total_variation_denoising). Observe the edge preserving effect in the denoised image inspite of smoothing. 

![](https://bitbucket.org/ga_learning/style_transfer/raw/48db127941e04f2f66cdfd6a3ccaca9ed888d598/markdown_images/TV_Denoising_Example.png)




In [1]:
def total_variation_loss(x):
    assert K.ndim(x) == 4
    if K.image_data_format() == 'channels_first':
        Gy = K.square(x[:, :, :img_h - 1, :img_w - 1] - x[:, :, 1:, :img_w - 1])
        Gx = K.square(x[:, :, :img_h - 1, :img_w - 1] - x[:, :, :img_h - 1, 1:])
    else:
        # Move the image pixel by pixel, and calculate the variance
        Gx = K.square(x[:, :img_h - 1, :img_w - 1, :] - x[:, 1:, :img_w - 1, :])
        Gy = K.square(x[:, :img_h - 1, :img_w - 1, :] - x[:, :img_h - 1, 1:, :])
    return K.sum(K.pow(Gx + Gy, 1.25))

Now the **total loss** can be reformulated using the total variance weight **gamma** as follows : 

### **L<sub>total</sub>**   =   alpha ***L<sub>content</sub>**  +  beta ***L<sub>style</sub>**   +  gamma * **L<sub>tv</sub>** 


In [6]:
def total_loss(Lc , Ls , Ltv , alpha = 0.5 , beta = 0.5 , gamma = 0.5):
    Lt = (alpha*Lc) + (beta*Ls) + (gamma*Ltv)
    return Lt

## Unit 6. Quiz 

Following are the quizzes for Chapter 3

### 1. Gram matrix plays an important role in calculating the following

    a. Style loss
    b. Content loss 
    c. Both a and b
    d. None of the above

### 2. Which of the following is controlled by the Total Variation Loss 

    a. Noise in the image 
    b. Smoothness of the output
    c. Both a and b
    d. None of the above 
    
### 3. Which of the following is true in the context of losses described in this chapter 

    a. Content loss controls the colour and textures in the output image  
    b. Style loss controls the colour and textures in the output image
    c. Both a and b 
    d. None of the above

### Answers :

**1. a ** 
   - Hint : Gram matrix helps identifying the features which get activated simultaneously, which is a good measure to compare the styles 

**2. c ** 
   - Hint : Total Variation Loss helps reducing the high frequency noises and also helps smoothing the image while preserving edges

**3. b ** 
   - Hint : Style loss deals with colours and texture . Content loss deals with edges, corners and objects